## Analyis of potential flare list

This notebook makes lots of plots and creates the final flare list of 18 events of interest.

Creates the `flare_ana_18_flares.csv` file

In [79]:
from sunpy.net import Fido, attrs as a
from stixpy.net.client import STIXClient
from sunpy import timeseries as ts
from sunpy.coordinates import get_horizons_coord, frames
from stixdcpy.net import JSONRequest as jreq
from sunpy.time import parse_time
import astropy.constants as const
from astropy import units as u
from astropy.coordinates import SkyCoord
from stixpy import timeseries
import matplotlib.pyplot as plt 
from matplotlib import dates
import numpy as np 
import pandas as pd
import datetime
import astrospice
import warnings
warnings.filterwarnings("ignore")

In [2]:
flares = pd.read_csv("stix_fermi_potential_flares_20210101_20220601_fixed.csv")

In [3]:
flares.reset_index(inplace=True, drop=True)

In [4]:
len(flares[flares["goes_class_ind"].isin(["X", "M"])])

48

In [5]:
flares.keys()

Index(['start_UTC', 'peak_UTC', 'end_UTC', 'duration', 'GOES_flux',
       'CFL_X_arcsec', 'CFL_Y_arcsec', 'GOES_class', 'LC0_peak_counts_4sec',
       'lon', 'rad', 'fermi', 'fermi_tstart', 'fermi_tpeak', 'fermi_tend',
       'fermi_peak_counts', 'fermi_total_counts', 'sun_det0', 'sun_det1',
       'sun_det2', 'sun_det3', 'goes_class_ind'],
      dtype='object')

In [6]:
flares_xm = flares[flares["goes_class_ind"].isin(["X", "M"])]

In [7]:
flares_xm.iloc[1]

start_UTC               2021-05-22 21:31:50.988
peak_UTC                2021-05-22T21:35:42.990
end_UTC                 2021-05-22T21:41:34.991
duration                                    584
GOES_flux                              0.000015
CFL_X_arcsec                             1050.0
CFL_Y_arcsec                              320.0
GOES_class                                 M1.5
LC0_peak_counts_4sec                      77823
lon                                  -98.139954
rad                            141546947.712483
fermi                                        24
fermi_tstart                2021-05-22 21:30:59
fermi_tpeak                 2021-05-22 21:31:46
fermi_tend                  2021-05-22 21:31:58
fermi_peak_counts                          7568
fermi_total_counts                       153646
sun_det0                                     n5
sun_det1                                     n1
sun_det2                                     n3
sun_det3                                

In [10]:
flares_xm["goes_class_ind"].unique()

array(['M', 'X'], dtype=object)

In [12]:
flares.iloc[0]

start_UTC               2021-05-07 18:51:07.200
peak_UTC                2021-05-07T19:00:15.200
end_UTC                 2021-05-07T19:18:55.202
duration                                   1668
GOES_flux                              0.000039
CFL_X_arcsec                              540.0
CFL_Y_arcsec                              270.0
GOES_class                                 M3.9
LC0_peak_counts_4sec                     311295
lon                                  -97.426482
rad                            137254463.509537
fermi                                        15
fermi_tstart                2021-05-07 18:58:45
fermi_tpeak                 2021-05-07 18:59:34
fermi_tend                  2021-05-07 19:35:36
fermi_peak_counts                         71659
fermi_total_counts                     17656436
sun_det0                                     n5
sun_det1                                     n3
sun_det2                                     n1
sun_det3                                

In [13]:


def make_plot(i):
    fl = flares.iloc[i]
    tstart_stix, tend_stix = fl["start_UTC"], fl["end_UTC"]
    tstart_fermi, tend_fermi = fl["fermi_tstart"], fl["fermi_tend"]
    det_fermi = fl["sun_det0"]
    
    
    tstart_plot = parse_time(tstart_stix).datetime - datetime.timedelta(minutes=5)
    tend_plot = parse_time(tend_stix).datetime  + datetime.timedelta(minutes=5)

    solo_pos = get_horizons_coord('SOLO', time=tstart_stix)
    earth_pos = get_horizons_coord('399', time=tstart_stix)
    light_tt = (solo_pos.radius - earth_pos.radius)/const.c
    light_tt.to('s')



    query_stix= Fido.search(a.Time(tstart_stix, tend_stix),
                            a.Instrument.stix,
                            a.stix.DataProduct.ql_lightcurve)

    query_fermi = Fido.search(a.Time(tstart_fermi, tend_fermi), 
                              a.Instrument.gbm,
                              a.Resolution.cspec,
                              a.Detector(det_fermi))

    files = Fido.fetch(query_stix, query_fermi, path="./{instrument}/{file}")

    if len(files)==2:
        if "STIX" in files[0]:
            stix_lc = ts.TimeSeries(files[0]).truncate(tstart_plot, tend_plot)
            gbm_lc = ts.TimeSeries(files[1]).truncate(tstart_plot, tend_plot)
        else:
            stix_lc = ts.TimeSeries(files[1]).truncate(tstart_plot, tend_plot)
            gbm_lc = ts.TimeSeries(files[2]).truncate(tstart_plot, tend_plot)        

    gbm_lc = gbm_lc.to_dataframe()
    stix_lc = stix_lc.to_dataframe()
    gbm_lc.replace(0, np.nan, inplace=True)    

    stix_lc.index = stix_lc.index - datetime.timedelta(seconds=light_tt.to_value(u.s))

    fig, (ax1, ax2) = plt.subplots(2, sharex=True, figsize=(6, 6))
    ax1.plot(stix_lc["4-10 keV"], label="4-10 keV")
    ax1.plot(stix_lc["10-15 keV"], label="10-15 keV")
    ax1.plot(stix_lc["15-25 keV"], label="15-25 keV")
    ax1.plot(stix_lc["25-50 keV"], label="25-50 keV")
    ax1.plot(stix_lc["50-84 keV"], label="50-84 keV")
    ax1.legend(loc="upper right")
    ax1.set_yscale("log")
    
    ax1.set_title("{:s} GOES class, {:.2f} degrees from Sun-Earth line".format(fl["GOES_class"], fl["lon"]))
    
    ax2.plot(gbm_lc["4-15 keV"], label="4-15 keV")
    ax2.plot(gbm_lc["15-25 keV"], label="15-25 keV")
    ax2.plot(gbm_lc["25-50 keV"], label="25-50 keV")
    ax2.plot(gbm_lc["50-100 keV"], label="50-100 keV")
    ax2.set_yscale("log")
    ax2.legend(loc="upper right")

    ax1.axvline(parse_time(tstart_stix).datetime, ls="dashed", lw=0.3)
    ax1.axvline(parse_time(tend_stix).datetime, ls="dashed", lw=0.3)
    
    ax1.xaxis.set_major_formatter(dates.DateFormatter("%H:%M"))
    ax1.set_ylabel("Counts/s/keV")
    ax2.set_ylabel("Counts/s/keV")

    ax1.text(0.02, 0.95, "a. STIX", transform=ax1.transAxes)
    ax2.text(0.02, 0.95, "b. Fermi/GBM", transform=ax2.transAxes)
    plt.tight_layout()
    ax2.set_xlabel("Time {:s} UT".format(stix_lc.index[0].strftime("%Y-%m-%d")))
    plt.savefig("./fermi_stix_plots/quick_look_{:s}_str{:d}_{:s}.png".format(parse_time(tstart_stix).strftime("%Y%m%d_%H%M%S"), i, fl["goes_class_ind"]))
    plt.close()

In [14]:
#make_plot(0)

In [15]:
def plot_all():
    errors = []
    for i in range(len(flares)):
        try:
            make_plot(i)
            print("doing", i)
        except:
            print("error", i)
            errors.append(i)

In [16]:
indices_of_interest = [17, 19, 20, 26, 35, 36, 47, 52, 146, 163, 166, 168, 181, 197, 198, 199, 203, 213]

In [17]:
flares_of_interest = flares.iloc[indices_of_interest]

In [18]:
flares_of_interest.head(3)

,start_UTC,peak_UTC,end_UTC,duration,GOES_flux,CFL_X_arcsec,CFL_Y_arcsec,GOES_class,LC0_peak_counts_4sec,lon,...,fermi_tstart,fermi_tpeak,fermi_tend,fermi_peak_counts,fermi_total_counts,sun_det0,sun_det1,sun_det2,sun_det3,goes_class_ind
17,2021-08-22 10:14:42.130,2021-08-22T10:16:22.130,2021-08-22T10:19:10.131,268,0.000002,700.0,310.0,C2.0,17407,-79.479501,...,2021-08-22 10:17:37,2021-08-22 10:18:14,2021-08-22 10:20:25,6733,210259,n5,n4,n2,n1,C
19,2021-08-28 05:00:36.576,2021-08-28T05:01:40.576,2021-08-28T05:03:04.576,148,0.000007,0.0,0.0,C7.0,118783,-72.405497,...,2021-08-28 05:03:28,2021-08-28 05:04:36,2021-08-28 05:06:06,16197,538835,n5,n3,n1,n4,C
20,2021-08-28 05:53:24.582,2021-08-28T06:04:24.583,2021-08-28T06:37:12.586,2628,0.000048,1070.0,-1220.0,M4.8,753663,-72.357369,...,2021-08-28 05:53:18,2021-08-28 06:06:22,2021-08-28 06:47:53,130560,62574028,n5,n1,n3,n0,M


In [19]:
#flares_of_interest.to_csv("18_flares_of_interest.csv")

## Plot location of SoLO and Earth

In [20]:
times_flares = parse_time(flares_of_interest["start_UTC"])

In [21]:
kernals = astrospice.registry.get_kernels("solar orbiter", "predict")
solo_coords = astrospice.generate_coords("SOLAR ORBITER", times_flares)
earth_coords = astrospice.generate_coords("earth", times_flares)
sun_coords = astrospice.generate_coords("sun", times_flares)

solo_coords_hgs = solo_coords.transform_to(frames.HeliographicStonyhurst)
earth_coords_hgs = earth_coords.transform_to(frames.HeliographicStonyhurst)
sun_coords_hgs = sun_coords.transform_to(frames.HeliographicStonyhurst)

Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

In [22]:
solo_coords_hgs[0]

<SkyCoord (HeliographicStonyhurst: obstime=2021-08-22 10:14:42.130, rsun=695700.0 km): (lon, lat, radius) in (deg, deg, km)
    (-79.47950134, -1.20182759, 96550261.10490535)>

In [23]:
ticks = dates.date2num(times_flares.datetime)

In [25]:
def make_orbit_plots():
    for i in range(len(flares_of_interest)):

        solo_pos = solo_coords_hgs[i]
        sun_pos = sun_coords_hgs[0]
        earth_pos = earth_coords_hgs[0]


        fig = plt.figure(figsize=(6, 6))
        ax = fig.add_subplot(projection='polar')


        ax.plot([solo_pos.lon.to_value(u.rad), sun_pos.lon.to_value(u.rad)], 
                [solo_pos.radius.to_value(u.AU), sun_pos.radius.to_value(u.AU)], color='k', ls="dashed")

        ax.plot([earth_pos.lon.to_value(u.rad), sun_pos.lon.to_value(u.rad)], 
                [earth_pos.radius.to_value(u.AU), sun_pos.radius.to_value(u.AU)], color='k', ls="dashed")

        ax.plot(solo_pos.lon.to(u.rad), 
                   solo_pos.radius.to_value(u.au), marker='.', ms=20, color='tab:orange', label="Solar Orbiter")

        ax.plot(earth_pos.lon.to(u.rad), 
                   earth_pos.radius.to_value(u.au), ms=12, marker='.', color="blue", label="Earth")

        ax.plot(sun_pos.lon.to(u.rad), 
                sun_pos.radius.to_value(u.AU), marker='o', color='y', label="Sun",  ms=20)



        ax.set_theta_zero_location("S")
        _ = ax.set_yticklabels([])

        ax.set_rlim(0, 1.3)
        ax.set_title("HGS Positions {:s} UT".format(solo_pos.obstime.strftime("%Y-%m-%d")))


        ax.legend()
        plt.tight_layout()
        plt.savefig("flares_of_interest/orbit_plot_{:d}.png".format(flares_of_interest.index[i]), dpi=300, 
                    facecolor='w', bbox_inches="tight")
        plt.close()

In [26]:
# fig = plt.figure(figsize=(6, 6))
# ax = fig.add_subplot(projection='polar')

# ax.plot((flares_of_interest.iloc[0]["lon"])*u.deg.to(u.rad), 
#        (flares_of_interest.iloc[0]["rad"]*u.km).to(u.AU), 
#         marker='.', ms=20)

# ax.plot([0], [1], marker='.', ms=20)
# ax.set_theta_zero_location("S")
# _ = ax.set_yticklabels([])

# ax.set_rlim(0, 1.3)
# ax.set_title("HGS Positions {:s} UT".format(solo_pos.obstime.strftime("%Y-%m-%d")))

In [27]:
len(flares_of_interest[["start_UTC", "GOES_class"]])

18

In [291]:
#flares_of_interest[["start_UTC", "GOES_class"]].to_csv("flares_to_get_location.csv")

In [28]:
flare_of_interest_w_locs = pd.read_csv("flares_to_get_location_w_locations.csv")

In [29]:
flares_of_interest["location_hgs"] = flare_of_interest_w_locs["location"].values

In [30]:
def make_orbit_plots():
    for i in range(len(flares_of_interest)):

        l = flares_of_interest.iloc[i]["location_hgs"]
        date = flares_of_interest.iloc[i]["start_UTC"]

        map_pos = {"S":-1, "N":1, "E":-1, "W":1}
        x = l[0:3]
        y = l[3:7]

        xx = map_pos[x[0]]*int(x[1:])
        yy = map_pos[y[0]]*int(y[1:])
        print(xx, yy)
        coord = SkyCoord(lat=xx*u.deg, lon=yy*u.deg, frame=frames.HeliographicStonyhurst)
        coord_hpc = coord.transform_to(frames.Helioprojective(observer="earth", obstime=date))
        coord_test = coord_hpc.transform_to(frames.HeliographicStonyhurst)

        solo_pos = solo_coords_hgs[i]
        sun_pos = sun_coords_hgs[0]
        earth_pos = earth_coords_hgs[0]


        fig = plt.figure(figsize=(6, 6))
        ax = fig.add_subplot(projection='polar')


        ax.plot([solo_pos.lon.to_value(u.rad), sun_pos.lon.to_value(u.rad)], 
                [solo_pos.radius.to_value(u.AU), sun_pos.radius.to_value(u.AU)], color='k', ls="dashed")

        ax.plot([earth_pos.lon.to_value(u.rad), sun_pos.lon.to_value(u.rad)], 
                [earth_pos.radius.to_value(u.AU), sun_pos.radius.to_value(u.AU)], color='k', ls="dashed")

        ax.plot(solo_pos.lon.to(u.rad), 
                   solo_pos.radius.to_value(u.au), marker='.', ms=20, color='tab:orange', label="Solar Orbiter")

        ax.plot(earth_pos.lon.to(u.rad), 
                   earth_pos.radius.to_value(u.au), ms=12, marker='.', color="blue", label="Earth")

        ax.plot(sun_pos.lon.to(u.rad), 
                sun_pos.radius.to_value(u.AU), marker='o', color='y', label="Sun",  ms=20)

        ax.plot(coord_test.lon.to(u.rad), 
                coord_test.radius.to_value(u.au)+0.08, marker='x', color='r')



        ax.set_theta_zero_location("S")
        _ = ax.set_yticklabels([])

        ax.set_rlim(0, 1.3)
        ax.set_title("HGS Positions {:s} UT".format(solo_pos.obstime.strftime("%Y-%m-%d")))


        ax.legend()
        plt.tight_layout()
        plt.savefig("flares_of_interest/orbit_plot_flare_{:d}.png".format(flares_of_interest.index[i]), dpi=300, 
                    facecolor='w', bbox_inches="tight")
        plt.close()

In [31]:
#make_orbit_plots()

In [62]:
def get_hpc(loc_hgs, date):
    map_pos = {"S":-1, "N":1, "E":-1, "W":1}
    x = loc_hgs[0:3]
    y = loc_hgs[3:7]

    xx = map_pos[x[0]]*int(x[1:])
    yy = map_pos[y[0]]*int(y[1:])

    coord = SkyCoord(lat=xx*u.deg, lon=yy*u.deg, frame=frames.HeliographicStonyhurst)
    coord_hpc = coord.transform_to(frames.Helioprojective(observer="earth", obstime=date))

    #print(coord_hpc.Tx.value, coord_hpc.Ty.value)
    
    return round(coord_hpc.Tx.value, 1), round(coord_hpc.Ty.value, 1)

In [71]:
location_hpc = np.array([get_hpc(flares_of_interest.iloc[i]["location_hgs"], flares_of_interest.iloc[i]["start_UTC"]) \
        for i in range(len(flares_of_interest))])

In [72]:
flares_of_interest["hpc_x"] = location_hpc[:, 0]
flares_of_interest["hpc_y"] = location_hpc[:, 1]

In [74]:
flares_of_interest[["start_UTC", "hpc_x", "hpc_y", "GOES_class"]]

,start_UTC,hpc_x,hpc_y,GOES_class
17,2021-08-22 10:14:42.130,-827.0,264.6,C2.0
19,2021-08-28 05:00:36.576,-85.4,-587.8,C7.0
20,2021-08-28 05:53:24.582,14.8,-534.7,M4.8
26,2021-09-22 20:08:40.073,-401.2,-524.5,C1.1
35,2021-10-26 02:44:51.577,-935.9,229.8,M1.3
36,2021-10-26 05:59:23.597,-931.7,246.1,C7.8
47,2021-10-28 10:27:27.418,0.0,-552.2,M2.1
52,2021-11-01 01:26:04.049,-332.3,637.7,M1.6
146,2022-02-03 20:39:24.549,823.8,300.3,C1.4
163,2022-03-02 17:28:48.149,-454.9,352.8,M2.0


In [80]:
earth_flare_solo_deg = []
for i in range(len(flares_of_interest)):
    flare_aux=jreq.request_flare_light_time_and_angle(utc=flares_of_interest.iloc[i]["start_UTC"], 
                                                     flare_x=flares_of_interest.iloc[i]["hpc_x"], 
                                                     flare_y=flares_of_interest.iloc[i]["hpc_y"], 
                                                     observer='stix')
    earth_flare_solo_deg.append(flare_aux["earth_flare_solo_deg"])

In [81]:
flares_of_interest["earth_flare_solo_deg"] = earth_flare_solo_deg

In [84]:
# flares_of_interest.to_csv("flare_ana_18_flares.csv")

In [83]:
flares_of_interest.head(2)

,start_UTC,peak_UTC,end_UTC,duration,GOES_flux,CFL_X_arcsec,CFL_Y_arcsec,GOES_class,LC0_peak_counts_4sec,lon,...,fermi_total_counts,sun_det0,sun_det1,sun_det2,sun_det3,goes_class_ind,location_hgs,hpc_x,hpc_y,earth_flare_solo_deg
17,2021-08-22 10:14:42.130,2021-08-22T10:16:22.130,2021-08-22T10:19:10.131,268,0.000002,700.0,310.0,C2.0,17407,-79.479501,...,210259,n5,n4,n2,n1,C,N19E67,-827.0,264.6,79.72396
19,2021-08-28 05:00:36.576,2021-08-28T05:01:40.576,2021-08-28T05:03:04.576,148,0.000007,0.0,0.0,C7.0,118783,-72.405497,...,538835,n5,n3,n1,n4,C,S31E06,-85.4,-587.8,72.83580
